In [2]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import seaborn as sns
from scipy import stats
import os
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")


In [6]:
# 读取数据

date_list_all = []
date_list_extra = []
data_folder = os.walk(r"../data")  
for path,dir_list,file_list in data_folder:  
    for file_name in file_list:  
        if file_name[:11] == "beijing_all" or file_name[:13] == "beijing_extra":
            path_now = os.path.join(path, file_name)
            
            try:
                data_now = pd.read_csv(path_now)
                a = data_now['date'] 
                print(path_now)
                if file_name[:11] == "beijing_all" :
                    date_list_all.append(path_now)
                elif file_name[:13] == "beijing_extra" :
                    date_list_extra.append(path_now)
            # 跳过异常编码的文件 eg: beijing_all_20141231.csv
            # 跳过异常内容文件 eg. beijing_all_20170703.csv -- 08
            except:
                pass

date_list_all.sort()
date_list_extra.sort()
data_all = pd.read_csv(date_list_all[0])
data_extra = pd.read_csv(date_list_extra[0])
for i in range(1,len(date_list_all)):
    print(date_list_all[i])
    data_now = pd.read_csv(date_list_all[i])
    data_all = pd.concat([data_all, data_now], axis=0)
for i in range(1,len(date_list_extra)):
    print(date_list_extra[i])
    data_now = pd.read_csv(date_list_extra[i])
    data_extra = pd.concat([data_extra, data_now], axis=0)
    
# 保存数据
data_all.to_csv("../data_all.csv")
data_extra.to_csv("../data_extra.csv")

In [ ]:
# 读取数据
data_all = pd.read_csv("../data_all.csv")
data_extra = pd.read_csv("../data_extra.csv")

# 删去无用的第一列
data_all = data_all.iloc[:,1:]
data_extra = data_extra.iloc[:,1:]

# 删去植物园数据
data_all = data_all.drop(columns=['植物园'])
data_extra = data_extra.drop(columns=['植物园'])

# 删去['PM2.5_24h','PM10_24h','SO2_24h','NO2_24h', 'O3_24h','CO_24h','PM10']
data_all = data_all[~(data_all['type']=='PM10_24h')]
data_all = data_all[~(data_all['type']=='PM2.5_24h')]
data_all = data_all[~(data_all['type']=='PM10')]
data_extra = data_extra[~(data_extra['type']=='SO2_24h')]
data_extra = data_extra[~(data_extra['type']=='NO2_24h')]
data_extra = data_extra[~(data_extra['type']=='CO_24h')]
data_extra = data_extra[~(data_extra['type']=='O3_24h')]

In [ ]:
# 将all数据和extra数据按时间合并
data_mix = pd.DataFrame()
for date_now in np.unique(data_all['date'].values): 
    df_date_all = data_all[data_all['date']==date_now]
    df_date_extra = data_extra[data_extra['date']==date_now]
    if not df_date_extra.shape[0]:
        continue
        
    for hour_now in np.unique(df_date_all['hour'].values):
        df_date_hour_all = df_date_all[df_date_all['hour']==hour_now]
        df_date_hour_extra = df_date_extra[df_date_extra['hour']==hour_now]
        if df_date_hour_extra.shape[0]==4 and df_date_hour_all.shape[0]==2:       
            df_mix_now = pd.concat([df_date_hour_all, df_date_hour_extra], axis=0)
            data_mix = pd.concat([data_mix, df_mix_now], axis=0)
            print(str(date_now)+' '+str(hour_now))
# 合并结果保存为data_mix文件
data_mix.to_csv("../data_mix.csv")
# 读取文件
data_mix = pd.read_csv("../data_mix.csv")
data_mix = data_mix.iloc[:,1:]

In [ ]:
# 各地数据分别清洗
def data_pass(args):
    return data_cleaning(*args)

def data_cleaning(data_mix, loc):
    print(loc)
    data_mix['flag'] = True
    for type_now in np.unique(data_mix['type'].values):
        list_type_mix = data_mix[data_mix['type']==type_now].index.tolist()
        if np.isnan(data_mix.iloc[list_type_mix[0],loc]):
            data_mix['flag'].iloc[list_type_mix[0]] = False
        if np.isnan(data_mix.iloc[list_type_mix[-1],loc]):
            data_mix['flag'].iloc[list_type_mix[-1]] = False
        for idx in range(1,len(list_type_mix)-1):
            if np.isnan(data_mix.iloc[list_type_mix[idx],loc]):
                mid = 0.5*(data_mix.iloc[list_type_mix[idx-1],loc]+data_mix.iloc[list_type_mix[idx+1],loc])
                if np.isnan(mid):
                    data_mix['flag'].iloc[list_type_mix[idx]] = False
                else:
                    data_mix.iloc[list_type_mix[idx],loc] = mid
                    
    for date_now in np.unique(data_mix['date'].values): 
        list_date_mix = data_mix[data_mix['date']==date_now].index.tolist()
        for hour_now in np.unique(data_mix.iloc[list_date_mix,1].values):
            list_hour_mix = data_mix.iloc[list_date_mix,1][data_mix.iloc[list_date_mix,1]==hour_now].index.tolist()
            if data_mix['flag'].iloc[list_hour_mix].sum() < 6:
                data_mix['flag'].iloc[list_hour_mix] = False
    
    dataset_now = data_mix[data_mix['flag']]    
    dataset_now = pd.concat([dataset_now.iloc[:,:3], dataset_now.iloc[:,loc]], axis=1)
    
    pca=PCA(n_components=1)
    data_pca = dataset_now[dataset_now['type']=='PM2.5'].iloc[:,0:3]
    data_pca.iloc[:,2] = 'PCA'
    typelist = np.unique(dataset_now['type'].values).tolist()
    feature_ori = np.zeros([dataset_now[dataset_now['type']=='PM2.5'].shape[0], len(typelist)])
    for idx in range(len(typelist)):
        feature_ori[:,idx] = dataset_now[dataset_now['type']==typelist[idx]].iloc[:,3].values
    feature_pca = pca.fit_transform(feature_ori)
    data_pca[dataset_now.columns.values[-1]]=feature_pca
    
    data_pca_plus = pd.DataFrame()
    for idx in range(data_pca.shape[0]):
        data_pca_plus = pd.concat([data_pca_plus, data_pca.iloc[idx:idx+1,:]], axis=0)
        data_pca_plus = pd.concat([data_pca_plus, dataset_now[dataset_now['type']=='PM2.5'].iloc[idx:idx+1,:]], axis=0)
    
    return dataset_now,data_pca_plus

In [ ]:
# 并行程序，对各地数据分别清洗
MultiNum = 17
pool = Pool(processes=MultiNum)
num_loc = data_mix.shape[1]-3
para_data_mix = [data_mix]*num_loc
para_loc = list(range(3,data_mix.shape[1]))
para_list = list(zip(para_data_mix,para_loc))
start = time.time()
dataset_loc_list = pool.map(data_pass,para_list)
end = time.time()
print('training lasted: '+str(end-start))

In [ ]:
# 保存清洗结果
for data_clean, data_clean_pca in dataset_loc_list:
    filename = '../data_mix_clean/'+data_clean.columns.values.tolist()[-1]+'.csv'
    data_clean.to_csv(filename)
    filename_pca = '../data_mix_clean_pca/'+data_clean_pca.columns.values.tolist()[-1]+'.csv'
    data_clean_pca.to_csv(filename_pca)
    
# 保存地名
np.save('../location.npy',data_mix.columns.values[3:], allow_pickle=True)

In [ ]:
# 各地数据一起清洗
def data_pass(args):
    return data_cleaning(*args)

def data_cleaning(data_mix, loc):
    print(loc)
    
    data_mix['flag'] = True
    for type_now in np.unique(data_mix['type'].values):
        list_type_mix = data_mix[data_mix['type']==type_now].index.tolist()
        if np.isnan(data_mix.iloc[list_type_mix[0],loc]):
            data_mix['flag'].iloc[list_type_mix[0]] = False
        if np.isnan(data_mix.iloc[list_type_mix[-1],loc]):
            data_mix['flag'].iloc[list_type_mix[-1]] = False
        for idx in range(1,len(list_type_mix)-1): 
            if np.isnan(data_mix.iloc[list_type_mix[idx],loc]):
                mid = 0.5*(data_mix.iloc[list_type_mix[idx-1],loc]+data_mix.iloc[list_type_mix[idx+1],loc])
                if np.isnan(mid):
                    data_mix['flag'].iloc[list_type_mix[idx]] = False
                else:
                    data_mix.iloc[list_type_mix[idx],loc] = mid
                    
    return data_mix

In [ ]:
# 并行程序，对各地数据一起进行数据清洗
MultiNum = 17
pool = Pool(processes=MultiNum)

num_loc = data_mix.shape[1]-3
para_data_mix = [data_mix]*num_loc
para_loc = list(range(3,data_mix.shape[1]))
para_list = list(zip(para_data_mix,para_loc))

start = time.time()
datamix_flag_list = pool.map(data_pass,para_list)
end = time.time()
print('training lasted: '+str(end-start))

pool.close() 
pool.join()

In [ ]:
flag_result = pd.DataFrame()
data_mix_filled = data_mix.copy()
for loc in para_loc:
    data_mix_filled.iloc[:,loc] = datamix_flag_list[loc-3].iloc[:,loc]
    flag_result[str(loc)] = datamix_flag_list[loc-3]['flag']
flag_result['flag_res'] = False
for idx in range(flag_result.shape[0]):
    if flag_result.iloc[idx,:].sum() == num_loc:
        flag_result['flag_res'].iloc[idx] = True
data_mix_filled['flag'] = flag_result['flag_res']

for date_now in np.unique(data_mix_filled['date'].values): 
    list_date_mix = data_mix_filled[data_mix_filled['date']==date_now].index.tolist()
    for hour_now in np.unique(data_mix_filled.iloc[list_date_mix,1].values):
        list_hour_mix = data_mix_filled.iloc[list_date_mix,1][data_mix_filled.iloc[list_date_mix,1]==hour_now].index.tolist()
        if data_mix_filled['flag'].iloc[list_hour_mix].sum() < 6:
            data_mix_filled['flag'].iloc[list_hour_mix] = False

dataset_now = data_mix_filled[data_mix_filled['flag']]    
dataset_now = dataset_now.drop(['flag'],axis=1)

# 保存文件
filename = 'data_mix_clean_all.csv'
dataset_now.to_csv(filename)